# 第18章：Advanced Streaming（高级流式输出）

## 学习目标

本章将学习：
1. 流式输出的核心价值
2. stream_mode参数详解（values、updates、messages、custom、debug）
3. Agent的流式输出
4. 多模式组合使用
5. 自定义流式更新（Custom Streaming）
6. astream_events()语义事件流
7. 流式输出的错误处理
8. 最佳实践和应用场景

---

## 为什么需要高级流式输出？

### 用户体验的关键

在AI应用中，LLM的响应时间可能需要几秒到几十秒，流式输出可以：

- ✅ **提升响应感**：用户看到实时输出，不会等待焦虑
- ✅ **显示进度**：展示Agent的思考过程和中间步骤
- ✅ **提前终止**：用户可以根据输出决定是否等待完成
- ✅ **调试友好**：开发者可以实时查看执行流程

### 基础流式 vs 高级流式

| 功能 | 基础stream() | 高级stream_mode + astream_events() |
|------|------------|----------------------------------|
| **Token流式** | ✅ 支持 | ✅ 支持 |
| **中间步骤** | ❌ 无 | ✅ 支持（updates模式） |
| **状态更新** | ❌ 无 | ✅ 支持（values模式） |
| **自定义数据** | ❌ 无 | ✅ 支持（custom模式） |
| **语义事件** | ❌ 无 | ✅ 支持（astream_events） |
| **多模式组合** | ❌ 无 | ✅ 支持 |

---

## 核心概念

### 1. stream_mode参数

LangGraph的`stream()`方法支持多种流式模式：

```python
agent.stream(
    {"messages": [...]},
    stream_mode="updates"  # 指定流式模式
)
```

**5种模式**：

| 模式 | 输出内容 | 使用场景 |
|------|---------|---------|
| `values` | 每步后的完整状态 | 需要完整State |
| `updates` | 每步的状态更新（增量） | Agent进度追踪 |
| `messages` | LLM tokens + 元数据 | 聊天应用 |
| `custom` | 自定义数据 | 工具进度反馈 |
| `debug` | 详细执行trace | 调试 |

### 2. astream_events()

语义事件流，提供更细粒度的控制：

```python
async for event in model.astream_events("Hello"):
    if event["event"] == "on_chat_model_stream":
        print(event["data"]["chunk"].text)
```

**事件类型**：
- `on_chat_model_start`：模型开始调用
- `on_chat_model_stream`：模型生成token
- `on_chat_model_end`：模型调用结束
- `on_tool_start`、`on_tool_end`：工具调用

### 3. 多模式组合

可以同时使用多种模式：

```python
stream_mode=["updates", "messages", "custom"]
# 输出格式：(mode, chunk)
```


In [1]:
# 环境配置
import os
import sys

_project_root = os.path.abspath(os.path.join(os.getcwd(), '..'))
sys.path.append(_project_root)

from config import config
from langchain_openai import ChatOpenAI
from langchain.agents import create_agent
from langgraph.checkpoint.memory import InMemorySaver
from langchain.tools import tool

# 初始化模型
model = ChatOpenAI(
    model="gpt-4.1-mini",
    temperature=0,
    api_key=config.CLOUD_API_KEY,
    base_url=config.CLOUD_BASE_URL,
)

print("环境配置完成！")
print(f"模型: {model.model_name}")

环境配置完成！
模型: gpt-4.1-mini


## 1. stream_mode="updates" - 追踪Agent步骤

### 核心价值

`updates`模式流式输出每个节点的状态更新（增量），适合追踪Agent的执行进度。

### 工作机制

```
Agent执行流程：
1. Agent节点：生成AIMessage（含tool_calls）
   → 输出：{"agent": {"messages": [AIMessage]}}

2. Tools节点：执行工具，返回ToolMessage
   → 输出：{"tools": {"messages": [ToolMessage]}}

3. Agent节点：生成最终AIMessage
   → 输出：{"agent": {"messages": [AIMessage]}}
```

每步只输出**增量更新**，而非完整状态。

In [3]:
print("【演示：stream_mode='updates'】\n")

# 创建一个简单工具
@tool
def search_web(query: str) -> str:
    """搜索网络获取信息"""
    import time
    time.sleep(0.5)  # 模拟网络延迟
    return f"关于'{query}'的搜索结果：这是一个很有趣的话题！"

# 创建Agent
agent = create_agent(
    model=model,
    tools=[search_web],
    system_prompt="你是一个助手，可以搜索信息"
)

# 使用stream_mode="updates"流式输出
for chunk in agent.stream(
    {"messages": [{"role": "user", "content": "搜索Python编程的最佳实践"}]},
    stream_mode="updates"  # 追踪每个节点的更新
):
    # chunk格式：{"node_name": {"messages": [...]}}
    for node_name, update in chunk.items():
        print(f"\n【节点：{node_name}】")
        messages = update.get("messages", [])
        for msg in messages:
            if hasattr(msg, 'content') and msg.content:
                print(f"  内容: {msg.content[:80]}...")
            elif hasattr(msg, 'tool_calls') and msg.tool_calls:
                print(f"  工具调用: {[tc['name'] for tc in msg.tool_calls]}")

print("\n✓ 完成！可以看到Agent的每一步执行过程")

【演示：stream_mode='updates'】


【节点：model】
  工具调用: ['search_web']

【节点：tools】
  内容: 关于'Python编程的最佳实践'的搜索结果：这是一个很有趣的话题！...

【节点：model】
  内容: Python编程的最佳实践通常包括以下几个方面：

1. 代码风格：遵循PEP 8编码规范，保持代码整洁和一致性。
2. 命名规范：变量、函数和类名应具有描述性...

✓ 完成！可以看到Agent的每一步执行过程


## 2. stream_mode="values" - 获取完整状态

### 核心价值

`values`模式流式输出每步之后的**完整状态**，而非增量更新。

### updates vs values

```python
# updates: 只输出增量
{"agent": {"messages": [新消息]}}

# values: 输出完整状态
{"messages": [消息1, 消息2, ..., 新消息]}
```

**适用场景**：
- ✅ 需要完整对话历史
- ✅ UI需要显示所有消息
- ✅ 需要当前完整状态

In [4]:
print("【演示：stream_mode='values'】\n")

# 使用stream_mode="values"流式输出完整状态
for chunk in agent.stream(
    {"messages": [{"role": "user", "content": "查询AI最新动态"}]},
    stream_mode="values"  # 获取完整状态
):
    # chunk格式：完整的state字典
    messages = chunk.get("messages", [])
    latest_message = messages[-1] if messages else None
    
    if latest_message:
        print(f"\n【状态更新】当前共{len(messages)}条消息")
        if hasattr(latest_message, 'content') and latest_message.content:
            print(f"  最新内容: {latest_message.content[:60]}...")
        elif hasattr(latest_message, 'tool_calls') and latest_message.tool_calls:
            print(f"  最新动作: 调用工具 {[tc['name'] for tc in latest_message.tool_calls]}")

print("\n✓ values模式输出完整状态，适合UI渲染")

【演示：stream_mode='values'】


【状态更新】当前共1条消息
  最新内容: 查询AI最新动态...

【状态更新】当前共2条消息
  最新动作: 调用工具 ['search_web']

【状态更新】当前共3条消息
  最新内容: 关于'AI最新动态 2024'的搜索结果：这是一个很有趣的话题！...

【状态更新】当前共4条消息
  最新内容: AI最新动态包括多个方面的发展，例如生成式AI技术的进步、AI在医疗、教育、金融等行业的应用扩展，以及各大科技公司在AI...

✓ values模式输出完整状态，适合UI渲染


## 3. stream_mode="messages" - 流式Token输出

### 核心价值

`messages`模式专门用于流式输出LLM生成的tokens，提供ChatGPT般的打字机效果。

### 输出格式

```python
# 每个chunk是一个2-tuple：(token, metadata)
(AIMessageChunk(content="你"), {"langgraph_node": "agent"})
(AIMessageChunk(content="好"), {"langgraph_node": "agent"})
(AIMessageChunk(content="！"), {"langgraph_node": "agent"})
```

**特点**：
- ✅ 实时token流式
- ✅ 包含元数据（来自哪个节点）
- ✅ 适合聊天UI

In [6]:
print("【演示：stream_mode='messages'】\n")

import sys

# 使用stream_mode="messages"流式输出tokens
print("Agent: ", end="", flush=True)

for chunk in agent.stream(
    {"messages": [{"role": "user", "content": "用200字介绍Python"}]},
    stream_mode="messages"  # 流式输出tokens
):
    # chunk格式：(AIMessageChunk, metadata)
    token, metadata = chunk
    
    if hasattr(token, 'content') and token.content:
        # 打印token（打字机效果）
        print(token.content, end="", flush=True)

print("\n\n✓ messages模式提供打字机效果，适合聊天应用")

【演示：stream_mode='messages'】

Agent: Python是一种高级编程语言，由Guido van Rossum于1991年发布。它以简洁易读的语法著称，适合初学者和专业开发者。Python支持多种编程范式，包括面向对象、函数式和过程式编程。它拥有丰富的标准库和第三方库，广泛应用于数据分析、人工智能、机器学习、网络开发、自动化脚本等领域。Python跨平台，支持Windows、macOS和Linux系统。其强大的社区支持和持续更新使其成为现代编程的重要工具。Python代码简洁，开发效率高，适合快速原型开发和大型项目。

✓ messages模式提供打字机效果，适合聊天应用


## 4. stream_mode="custom" - 自定义流式更新

### 核心价值

`custom`模式允许从工具内部发送自定义进度更新，提供更细粒度的用户反馈。

### 使用方法

在工具中使用`runtime.stream_writer`发送自定义数据：

```python
from langchain.tools import tool, ToolRuntime

@tool
def process_data(data: str, runtime: ToolRuntime) -> str:
    writer = runtime.stream_writer
    
    writer("步骤1：正在加载数据...")
    # 执行步骤1
    
    writer("步骤2：正在处理数据...")
    # 执行步骤2
    
    writer("步骤3：正在保存结果...")
    # 执行步骤3
    
    return "完成！"
```

**适用场景**：
- ✅ 长时间工具调用
- ✅ 多步骤处理
- ✅ 进度反馈

In [10]:
print("【演示：stream_mode='custom'】\n")

from langchain.tools import ToolRuntime
import time

# 创建带自定义流式更新的工具
@tool
def analyze_code(code: str, runtime: ToolRuntime) -> str:
    """分析代码质量"""
    writer = runtime.stream_writer
    
    writer("📥 正在加载代码...")
    time.sleep(0.3)
    
    writer("🔍 正在检查语法...")
    time.sleep(0.3)
    
    writer("⚡ 正在分析性能...")
    time.sleep(0.3)
    
    writer("🛡️ 正在检查安全性...")
    time.sleep(0.3)
    
    return "代码质量良好，无明显问题。"

# 创建带自定义工具的Agent
agent_custom = create_agent(
    model=model,
    tools=[analyze_code],
    system_prompt="你是代码分析助手"
)

# 使用stream_mode="custom"接收自定义更新
for chunk in agent_custom.stream(
    {"messages": [{"role": "user", "content": "分析我的Python代码，必须调用analyze_code工具"}]},
    stream_mode="custom"  # 接收自定义流式更新
):
    print(f"  {chunk}")

print("\n✓ custom模式允许工具发送进度更新")

【演示：stream_mode='custom'】

  📥 正在加载代码...
  🔍 正在检查语法...
  ⚡ 正在分析性能...
  🛡️ 正在检查安全性...

✓ custom模式允许工具发送进度更新


## 4.5 stream_mode="debug" - 详细调试信息

### 核心价值

`debug`模式输出最详细的执行信息，包括节点执行、状态变化、时间戳等，专为调试设计。

### 输出内容

Debug模式会输出：
- ✅ 节点执行的详细信息
- ✅ 每个步骤的输入输出
- ✅ 时间戳和执行时长
- ✅ 状态变化的完整trace
- ✅ 错误和警告信息

### 适用场景

- 🐛 **开发调试**：排查Agent执行问题
- 🔍 **性能分析**：找出性能瓶颈
- 📊 **执行审计**：记录完整执行过程
- ⚠️ **错误诊断**：定位错误发生位置

**注意**：debug模式输出信息量大，**仅用于开发环境**，生产环境不建议使用。

In [15]:
print("【演示：stream_mode='debug'】\n")

# 使用stream_mode="debug"查看详细执行信息
print("开始执行（debug模式会输出大量信息）...\n")

chunk_count = 0
for chunk in agent.stream(
    {"messages": [{"role": "user", "content": "搜索Python最佳实践"}]},
    stream_mode="debug"  # 详细调试信息
):
    chunk_count += 1
    
    # debug模式的chunk是一个字典，包含丰富的调试信息
    if chunk_count <= 3:  # 只显示前3个chunk，避免输出过多
        print(f"【Debug Chunk {chunk_count}】")
        print(f"  类型: {chunk.get('type', 'N/A')}")
        print(f"  时间戳: {chunk.get('timestamp', 'N/A')}")
        
        # 显示payload的关键信息
        payload = chunk.get('payload', {})
        if 'task' in payload:
            task = payload['task']
            print(f"  任务ID: {task.get('id', 'N/A')}")
            print(f"  节点: {task.get('name', 'N/A')}")
        
        print()

print(f"✓ debug模式共输出 {chunk_count} 个调试chunks")
print("✓ debug模式适合开发调试，可以看到完整的执行trace")
print("⚠️  生产环境不建议使用debug模式（信息量大，影响性能）")

【演示：stream_mode='debug'】

开始执行（debug模式会输出大量信息）...

【Debug Chunk 1】
  类型: task
  时间戳: 2025-12-22T16:00:21.882022+00:00

【Debug Chunk 2】
  类型: task_result
  时间戳: 2025-12-22T16:00:23.525768+00:00

【Debug Chunk 3】
  类型: task
  时间戳: 2025-12-22T16:00:23.526038+00:00

✓ debug模式共输出 6 个调试chunks
✓ debug模式适合开发调试，可以看到完整的执行trace
⚠️  生产环境不建议使用debug模式（信息量大，影响性能）


## 5. 多模式组合 - 最佳实践

### 核心价值

可以同时使用多个`stream_mode`，获得更丰富的流式信息。

### 输出格式

当使用多模式时，每个chunk变为`(mode, data)`元组：

```python
stream_mode=["updates", "messages", "custom"]

# 输出：
("updates", {"agent": {"messages": [...]}})
("messages", (AIMessageChunk(...), metadata))
("custom", "自定义进度信息")
```

### 推荐组合

| 场景 | 推荐组合 | 原因 |
|------|---------|------|
| **聊天应用** | `["updates", "messages"]` | 既有token流式，又能追踪步骤 |
| **带工具的聊天** | `["updates", "messages", "custom"]` | 完整的进度反馈 |
| **调试** | `["updates", "debug"]` | 详细执行信息 |
| **UI渲染** | `["values", "messages"]` | 完整状态+token流式 |

In [11]:
print("【演示：多模式组合】\n")

# 使用多个stream_mode
for mode, chunk in agent_custom.stream(
    {"messages": [{"role": "user", "content": "分析一段代码，必须调用analyze_code工具"}]},
    stream_mode=["updates", "messages", "custom"]  # 组合使用3种模式
):
    if mode == "updates":
        # Agent步骤更新
        for node_name in chunk.keys():
            print(f"  [updates] 节点执行: {node_name}")
    
    elif mode == "messages":
        # LLM token流式
        token, metadata = chunk
        if hasattr(token, 'content') and token.content:
            print(f"  [messages] Token: {token.content[:20]}...", end="")
    
    elif mode == "custom":
        # 自定义进度
        print(f"  [custom] {chunk}")

print("\n\n✓ 多模式组合提供最丰富的流式信息")

【演示：多模式组合】

  [updates] 节点执行: model
  [custom] 📥 正在加载代码...
  [custom] 🔍 正在检查语法...
  [custom] ⚡ 正在分析性能...
  [custom] 🛡️ 正在检查安全性...
  [messages] Token: 代码质量良好，无明显问题。...  [updates] 节点执行: tools
  [messages] Token: 这...  [messages] Token: 段...  [messages] Token: 代码...  [messages] Token: 质量...  [messages] Token: 良...  [messages] Token: 好...  [messages] Token: ，没有...  [messages] Token: 明显...  [messages] Token: 的问题...  [messages] Token: 。...  [messages] Token: 它...  [messages] Token: 定义...  [messages] Token: 了...  [messages] Token: 一个...  [messages] Token: 简单...  [messages] Token: 的...  [messages] Token: 加...  [messages] Token: 法...  [messages] Token: 函数...  [messages] Token: ，并...  [messages] Token: 正确...  [messages] Token: 地...  [messages] Token: 调用...  [messages] Token: 和...  [messages] Token: 输出...  [messages] Token: 结果...  [messages] Token: 。如果...  [messages] Token: 你...  [messages] Token: 有...  [messages] Token: 具体...  [messages] Token: 的...  [messages] Token: 代码...  [messages] Token: 需要

## 6. astream_events() - 语义事件流

### 核心价值

`astream_events()`提供基于语义的事件流，可以精确过滤特定类型的事件。

### 主要事件类型

| 事件类型 | 触发时机 | 数据内容 |
|---------|---------|---------|
| `on_chat_model_start` | LLM开始调用 | 输入消息 |
| `on_chat_model_stream` | LLM生成token | token chunk |
| `on_chat_model_end` | LLM调用结束 | 完整输出 |
| `on_tool_start` | 工具开始执行 | 工具输入 |
| `on_tool_end` | 工具执行结束 | 工具输出 |
| `on_chain_start` | Chain开始 | 输入数据 |
| `on_chain_end` | Chain结束 | 输出数据 |

### 工作机制

```python
async for event in model.astream_events("Hello"):
    event_type = event["event"]  # 事件类型
    event_data = event["data"]   # 事件数据
    metadata = event["metadata"] # 元数据
```

**优势**：
- ✅ 精确的事件过滤
- ✅ 自动聚合完整消息
- ✅ 适合复杂应用

In [12]:
print("【演示：astream_events()】\n")

import asyncio

async def demo_astream_events():
    """演示astream_events的使用"""
    
    # 注意：astream_events是异步的
    async for event in model.astream_events("Python是什么？"):
        event_type = event["event"]
        
        if event_type == "on_chat_model_start":
            print(f"  ▶️ LLM开始调用")
        
        elif event_type == "on_chat_model_stream":
            # 流式token
            chunk = event["data"]["chunk"]
            if hasattr(chunk, 'content') and chunk.content:
                print(chunk.content, end="", flush=True)
        
        elif event_type == "on_chat_model_end":
            print(f"\n  ✅ LLM调用完成")
            # 可以获取完整输出
            # output = event["data"]["output"]
    
    print("\n✓ astream_events提供精确的事件控制")

# 运行异步函数
await demo_astream_events()

【演示：astream_events()】

  ▶️ LLM开始调用
Python是一种高级编程语言，具有简洁易读的语法和强大的功能。它由Guido van Rossum于1991年首次发布，设计目标是提高代码的可读性和开发效率。Python支持多种编程范式，包括面向对象编程、过程式编程和函数式编程。

Python的主要特点包括：

1. **简洁易读**：代码风格清晰，使用缩进来表示代码块，减少了代码的复杂性。
2. **丰富的标准库**：内置大量模块，支持文件操作、网络通信、数据处理等多种功能。
3. **跨平台**：可以运行在Windows、macOS、Linux等多种操作系统上。
4. **动态类型**：变量类型在运行时确定，灵活方便。
5. **广泛应用**：适用于Web开发、数据分析、人工智能、自动化脚本、科学计算等领域。

由于这些优点，Python成为了初学者和专业开发者都非常喜欢使用的编程语言。
  ✅ LLM调用完成

✓ astream_events提供精确的事件控制


## 7. 实战案例：构建完整的流式聊天应用

### 应用场景

构建一个类似ChatGPT的聊天应用，需要：
1. 实时显示LLM token（打字机效果）
2. 显示Agent的思考过程
3. 显示工具调用进度
4. 错误处理

### 完整实现

In [13]:
print("【实战：完整的流式聊天应用】\n")

# 1. 创建工具（带进度反馈）
@tool
def fetch_weather(city: str, runtime: ToolRuntime) -> str:
    """获取城市天气"""
    writer = runtime.stream_writer
    
    writer(f"🌍 正在连接天气API...")
    time.sleep(0.3)
    
    writer(f"🔍 正在查询{city}的天气...")
    time.sleep(0.3)
    
    writer(f"📊 正在解析数据...")
    time.sleep(0.3)
    
    return f"{city}今天晴天，温度25°C"

@tool
def calculate(expression: str, runtime: ToolRuntime) -> str:
    """计算数学表达式"""
    writer = runtime.stream_writer
    writer(f"🧮 正在计算: {expression}")
    time.sleep(0.2)
    
    try:
        result = eval(expression)
        return f"计算结果：{result}"
    except Exception as e:
        return f"计算错误：{str(e)}"

# 2. 创建Agent
chat_agent = create_agent(
    model=model,
    tools=[fetch_weather, calculate],
    system_prompt="你是一个友好的助手，可以查询天气和计算数学"
)

# 3. 流式聊天函数
def stream_chat(user_message: str):
    """流式输出聊天响应"""
    print(f"\n👤 用户: {user_message}")
    print("🤖 助手: ", end="", flush=True)
    
    current_content = ""
    
    for mode, chunk in chat_agent.stream(
        {"messages": [{"role": "user", "content": user_message}]},
        stream_mode=["updates", "messages", "custom"]
    ):
        if mode == "updates":
            # 显示节点执行
            for node_name in chunk.keys():
                if node_name == "tools":
                    print(f"\n   [🔧 执行工具]", end="", flush=True)
        
        elif mode == "messages":
            # 流式显示token
            token, _ = chunk
            if hasattr(token, 'content') and token.content:
                current_content += token.content
                print(token.content, end="", flush=True)
        
        elif mode == "custom":
            # 显示自定义进度
            print(f"\n   [{chunk}]", end="", flush=True)
    
    print()  # 换行
    return current_content

# 4. 测试多轮对话
queries = [
    "北京的天气怎么样？",
    "计算 123 * 456",
]

for query in queries:
    response = stream_chat(query)
    print()

print("✓ 完整的流式聊天应用演示完成")

【实战：完整的流式聊天应用】


👤 用户: 北京的天气怎么样？
🤖 助手: 
   [🌍 正在连接天气API...]
   [🔍 正在查询北京的天气...]
   [📊 正在解析数据...]北京今天晴天，温度25°C
   [🔧 执行工具]北京今天是晴天，温度大约25摄氏度。需要了解其他城市的天气吗？


👤 用户: 计算 123 * 456
🤖 助手: 
   [🧮 正在计算: 123 * 456]计算结果：56088
   [🔧 执行工具]123 * 456 的计算结果是 56088。

✓ 完整的流式聊天应用演示完成


## 8. 流式输出的错误处理

### 常见问题

流式输出可能遇到的问题：

1. **中断异常**：流式输出过程中发生错误
2. **部分输出**：只收到部分数据
3. **超时**：长时间无响应

### 错误处理策略

```python
try:
    for chunk in agent.stream(...):
        # 处理chunk
        pass
except Exception as e:
    print(f"流式输出错误：{e}")
    # 回退到非流式
    response = agent.invoke(...)
```

### 最佳实践

1. **超时控制**：设置合理的超时时间
2. **异常捕获**：捕获并处理流式异常
3. **回退机制**：流式失败时回退到`invoke()`
4. **用户提示**：告知用户当前状态

In [14]:
print("【演示：错误处理】\n")

def safe_stream_chat(user_message: str, timeout=30):
    """带错误处理的流式聊天"""
    print(f"用户: {user_message}")
    print("助手: ", end="", flush=True)
    
    try:
        full_response = ""
        
        for mode, chunk in chat_agent.stream(
            {"messages": [{"role": "user", "content": user_message}]},
            stream_mode=["messages"],
            # 可以通过config设置超时等参数
        ):
            if mode == "messages":
                token, _ = chunk
                if hasattr(token, 'content') and token.content:
                    full_response += token.content
                    print(token.content, end="", flush=True)
        
        print()
        return full_response
    
    except KeyboardInterrupt:
        print("\n\n⚠️ 用户中断")
        return None
    
    except Exception as e:
        print(f"\n\n❌ 流式输出错误: {e}")
        print("   正在尝试非流式模式...")
        
        try:
            # 回退到invoke()
            response = chat_agent.invoke(
                {"messages": [{"role": "user", "content": user_message}]}
            )
            result = response['messages'][-1].content
            print(f"   ✓ 回退成功: {result[:50]}...")
            return result
        except Exception as e2:
            print(f"   ❌ 回退也失败: {e2}")
            return None

# 测试
response = safe_stream_chat("你好")
print("\n✓ 带错误处理的流式输出更健壮")

【演示：错误处理】

用户: 你好
助手: 你好！有什么我可以帮忙的吗？

✓ 带错误处理的流式输出更健壮


## 9. 总结与最佳实践

### stream_mode选择指南

| 应用场景 | 推荐模式 | 原因 |
|---------|---------|------|
| **简单聊天** | `messages` | 只需token流式 |
| **Agent聊天** | `["updates", "messages"]` | 追踪步骤+token流式 |
| **带工具Agent** | `["updates", "messages", "custom"]` | 完整进度反馈 |
| **状态展示** | `["values", "messages"]` | 完整状态+token流式 |
| **调试开发** | `["updates", "debug"]` | 详细执行信息 |
| **仅进度追踪** | `updates` | 只关注步骤 |
| **自定义反馈** | `custom` | 工具内部进度 |

### 核心最佳实践

#### 1. 根据需求选择模式

```python
# ✅ 好的做法：根据场景选择
if is_chat_app:
    stream_mode = ["messages", "updates"]
elif is_debug_mode:
    stream_mode = ["debug"]
else:
    stream_mode = ["values"]

# ❌ 坏的做法：总是使用所有模式
stream_mode = ["values", "updates", "messages", "custom", "debug"]  # 过多
```

#### 2. 错误处理

```python
# ✅ 好的做法：捕获异常
try:
    for chunk in agent.stream(...):
        process(chunk)
except Exception as e:
    fallback_to_invoke()

# ❌ 坏的做法：不处理异常
for chunk in agent.stream(...):  # 可能崩溃
    process(chunk)
```

#### 3. 用户体验优化

```python
# ✅ 好的做法：提供进度反馈
@tool
def long_running_task(data: str, runtime: ToolRuntime) -> str:
    writer = runtime.stream_writer
    
    writer("步骤1：开始处理...")
    # 处理
    writer("步骤2：正在分析...")
    # 分析
    writer("步骤3：完成！")
    
    return result

# ❌ 坏的做法：无进度反馈
@tool
def long_running_task(data: str) -> str:
    # 长时间处理，用户不知道发生了什么
    time.sleep(10)
    return result
```

#### 4. 性能考虑

```python
# ✅ 好的做法：合理buffer
collected_tokens = []
for token in stream:
    collected_tokens.append(token)
    if len(collected_tokens) >= 10:  # 每10个token更新一次UI
        update_ui(''.join(collected_tokens))
        collected_tokens = []

# ❌ 坏的做法：每个token都更新UI
for token in stream:
    update_ui(token)  # 可能造成UI闪烁
```

---

### 常见问题

#### Q1: 什么时候使用stream()，什么时候使用astream_events()？

**stream()**：适合大多数场景，简单直接
- ✅ 聊天应用
- ✅ Agent进度追踪
- ✅ 自定义进度反馈

**astream_events()**：适合需要精确控制的场景
- ✅ 复杂的事件过滤
- ✅ 需要区分不同组件的输出
- ✅ 高级调试

#### Q2: updates和values有什么区别？

- **updates**：增量（delta），每步只输出新增的数据
- **values**：完整状态，每步输出整个state

选择原则：
- 需要完整历史 → `values`
- 只关注新变化 → `updates`

#### Q3: 多模式会影响性能吗？

会有轻微影响，但通常可以忽略。建议：
- 开发调试：使用多模式
- 生产环境：只使用必需的模式

#### Q4: 如何在流式输出时保存完整消息？

```python
# 方法1：手动聚合
full_message = ""
for mode, chunk in agent.stream(..., stream_mode=["messages"]):
    token, _ = chunk
    full_message += token.content

# 方法2：使用astream_events
# astream_events会在后台自动聚合
async for event in model.astream_events(...):
    if event["event"] == "on_chat_model_end":
        full_message = event["data"]["output"]
```

#### Q5: 可以中途取消流式输出吗？

可以，直接`break`退出循环：

```python
for chunk in agent.stream(...):
    if should_stop():
        break  # 停止接收
    process(chunk)
```

---

### 进阶话题

#### 1. 自定义流式处理器

```python
class StreamHandler:
    def __init__(self):
        self.tokens = []
    
    def on_token(self, token):
        self.tokens.append(token)
        # 自定义处理逻辑
    
    def on_complete(self):
        return ''.join(self.tokens)

handler = StreamHandler()
for mode, chunk in agent.stream(..., stream_mode=["messages"]):
    if mode == "messages":
        token, _ = chunk
        handler.on_token(token.content)

result = handler.on_complete()
```

#### 2. WebSocket实时推送

```python
# 伪代码：在Web应用中使用流式输出
async def stream_to_websocket(websocket, user_message):
    async for mode, chunk in agent.astream(...):
        if mode == "messages":
            token, _ = chunk
            await websocket.send_json({
                "type": "token",
                "content": token.content
            })
        elif mode == "custom":
            await websocket.send_json({
                "type": "progress",
                "message": chunk
            })
```

---

## 下一步学习

完成高级流式输出学习后，建议学习：

1. **第19章：异步与并发** - 异步编程和性能优化
2. **第20章：生产部署** - 监控、日志、扩展
3. **完整的生产级应用** - 整合所有技术

---

## 关键收获

✅ **stream_mode提供5种模式**：values、updates、messages、custom、debug

✅ **多模式组合**：`stream_mode=["updates", "messages", "custom"]`

✅ **自定义进度**：使用`runtime.stream_writer`发送自定义更新

✅ **astream_events()**：提供语义事件流，适合复杂场景

✅ **错误处理**：捕获异常，提供回退机制

✅ **用户体验**：流式输出大幅提升响应感

✅ **根据场景选择模式**：聊天用messages，调试用debug，完整应用用组合

✅ **工具内进度反馈**：长时间工具调用必须提供进度信息